# `atmchile` — Guía de Uso y Análisis de Disponibilidad de Datos

**[atmchile](https://github.com/maigonzalezh/py-atmchile)** es una librería Python para descargar datos de calidad del aire y clima desde dos redes de monitoreo del gobierno de Chile:

- **SINCA** (Sistema de Información Nacional de Calidad del Aire) — operado por el Ministerio del Medio Ambiente (MMA)
- **DMC** (Dirección Meteorológica de Chile) — operado por la DGAC

La librería retorna DataFrames de pandas con columnas de linaje de datos (`dl.*`) que permiten auditar la calidad y disponibilidad de cada valor descargado.

En este notebook:
1. Exploramos las estaciones disponibles
2. Descargamos datos de calidad del aire (sync y async)
3. Calculamos el **Data Capture Rate (DCR)** como métrica de disponibilidad por estación y región
4. Descargamos datos climáticos del DMC

> **Periodo de análisis:** Julio 2024 (invierno — episodios de contaminación por PM2.5 y patrones de disponibilidad de interés)

## 1. Setup

In [1]:
!pip install -q atmchile plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 10.5 MB/s eta 0:00:00


In [2]:
import asyncio
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from atmchile import ChileAirQuality, ChileClimateData

# Periodo de análisis
START = datetime(2024, 7, 1)
END = datetime(2024, 7, 31)

print(f"atmchile ready — periodo: {START:%Y-%m-%d} a {END:%Y-%m-%d}")

atmchile ready — periodo: 2024-07-01 a 2024-07-31


## 2. Explorar estaciones disponibles

Antes de descargar datos, veamos qué estaciones tenemos. La tabla de estaciones viene incluida en la librería y se puede actualizar con `uv run refresh-stations`.

In [3]:
caq = ChileAirQuality()
stations = caq.stations_table.copy()

print(f"Total de estaciones SINCA: {len(stations)}")
print(f"Columnas: {stations.columns.tolist()}")
stations.head()

Total de estaciones SINCA: 123
Columnas: ['city', 'station_code', 'latitude', 'longitude', 'station_name', 'region', 'network', 'region_index', 'access_type', 'operator']


,city,station_code,latitude,longitude,station_name,region,network,region_index,access_type,operator
0,Alto Hospicio,RI/117,-20.290467,-70.100192,Alto Hospicio,I,Red MMA,2,Pública,Ministerio del Medio Ambiente
1,Tocopilla,RII/201,-22.093082,-70.201210,Gobernación,II,Ciudad Tocopilla,3,Privada,Norgener S.A
2,Mejillones,RII/209,-23.098798,-70.442765,Jardín Infantil Integra,II,Red ENAEX,3,no definida,ENAEX S.A. Planta Prillex América
3,Calama,RII/217,-22.454016,-68.910148,Hospital del Cobre,II,NaN,3,Privada,Codelco Distrito Norte
4,Tocopilla,RII/230,-22.111064,-70.210828,Bomberos,II,Ciudad Tocopilla,3,Privada,Norgener S.A


In [4]:
# Conteo de estaciones por región
region_counts = (
    stations.groupby("region")
    .size()
    .reset_index(name="n_estaciones")
    .sort_values("n_estaciones", ascending=True)
)

fig = px.bar(
    region_counts,
    x="n_estaciones",
    y="region",
    orientation="h",
    title="Estaciones SINCA por región",
    labels={"n_estaciones": "N° de estaciones", "region": "Región"},
    text="n_estaciones",
)
fig.update_layout(height=500, showlegend=False)
fig.show()

In [5]:
# Mapa de estaciones
fig = px.scatter_mapbox(
    stations,
    lat="latitude",
    lon="longitude",
    color="region",
    hover_name="station_name",
    hover_data=["city", "network", "operator"],
    title="Red de monitoreo SINCA",
    zoom=3,
    height=600,
)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

## 3. Descarga básica — una estación

Empecemos con un caso simple: PM2.5 y PM10 para una estación de Santiago (RM/D14) durante julio 2024.

In [8]:
df_single = caq.get_data(
    stations=["RM/D14"],
    parameters=["PM25", "PM10"],
    start=START,
    end=END,
)

print(f"Shape: {df_single.shape}")
print(f"Columnas: {df_single.columns.tolist()}")
df_single.head(10)

Shape: (743, 9)
Columnas: ['date', 'city', 'station_code', 'station_name', 'region', 'PM25', 'dl.PM25', 'PM10', 'dl.PM10']


,date,city,station_code,station_name,region,PM25,dl.PM25,PM10,dl.PM10
0,2024-07-01 01:00:00,Santiago,RM/D14,Parque O'Higgins,RM,45.0,ok,86.0,ok
1,2024-07-01 02:00:00,Santiago,RM/D14,Parque O'Higgins,RM,42.0,ok,82.0,ok
2,2024-07-01 03:00:00,Santiago,RM/D14,Parque O'Higgins,RM,34.0,ok,62.0,ok
3,2024-07-01 04:00:00,Santiago,RM/D14,Parque O'Higgins,RM,29.0,ok,51.0,ok
4,2024-07-01 05:00:00,Santiago,RM/D14,Parque O'Higgins,RM,23.0,ok,42.0,ok
5,2024-07-01 06:00:00,Santiago,RM/D14,Parque O'Higgins,RM,20.0,ok,56.0,ok
6,2024-07-01 07:00:00,Santiago,RM/D14,Parque O'Higgins,RM,29.0,ok,132.0,ok
7,2024-07-01 08:00:00,Santiago,RM/D14,Parque O'Higgins,RM,33.0,ok,110.0,ok
8,2024-07-01 09:00:00,Santiago,RM/D14,Parque O'Higgins,RM,31.0,ok,107.0,ok
9,2024-07-01 10:00:00,Santiago,RM/D14,Parque O'Higgins,RM,26.0,ok,78.0,ok


### Columnas de linaje (`dl.*`)

Cada parámetro descargado viene acompañado de una columna `dl.{param}` que indica el status de cada valor:

| Valor | Significado |
|-------|-------------|
| `ok` | Dato descargado y presente |
| `empty` | La fila existe pero no se reportó medición |
| `download_error` | Falló la descarga HTTP |
| `curated` | Dato presente pero eliminado por inconsistencia física |

In [9]:
# Distribución de status de descarga
for param in ["PM25", "PM10"]:
    col = f"dl.{param}"
    if col in df_single.columns:
        print(f"\n{col}:")
        print(df_single[col].value_counts())


dl.PM25:
dl.PM25
ok       740
empty      3
Name: count, dtype: int64

dl.PM10:
dl.PM10
ok       739
empty      4
Name: count, dtype: int64


In [10]:
# Serie temporal PM2.5 y PM10
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    subplot_titles=("PM2.5 (µg/m³N)", "PM10 (µg/m³N)"),
    vertical_spacing=0.08,
)

if "PM25" in df_single.columns:
    fig.add_trace(
        go.Scatter(x=df_single["date"], y=df_single["PM25"],
                   mode="lines", name="PM2.5",
                   line=dict(color="#e74c3c", width=1)),
        row=1, col=1,
    )

if "PM10" in df_single.columns:
    fig.add_trace(
        go.Scatter(x=df_single["date"], y=df_single["PM10"],
                   mode="lines", name="PM10",
                   line=dict(color="#3498db", width=1)),
        row=2, col=1,
    )

fig.update_layout(
    title="Parque O'Higgins — Julio 2024",
    height=500,
    showlegend=True,
)
fig.show()

## 4. Descarga regional con async

Para el análisis de disponibilidad necesitamos datos de múltiples estaciones. Usamos `get_data_async()` con `region=True` para descargar regiones completas en paralelo.

Descargaremos PM2.5 de 4 regiones representativas: **RM** (Santiago), **V** (Valparaíso), **VIII** (Biobío) y **II** (Antofagasta).

In [11]:
TARGET_REGIONS = ["RM", "V", "VIII", "II"]

async def download_regions():
    caq_async = ChileAirQuality(max_concurrent_downloads=10)
    df = await caq_async.get_data_async(
        stations=TARGET_REGIONS,
        parameters=["PM25"],
        start=START,
        end=END,
        region=True,
    )
    return df

df_regions = await download_regions()

print(f"Shape: {df_regions.shape}")
print(f"Estaciones descargadas: {df_regions['station_name'].nunique()}")
print(f"Regiones: {sorted(df_regions['region'].unique())}")
df_regions.head()

Shape: (52753, 7)
Estaciones descargadas: 71
Regiones: ['II', 'RM', 'V', 'VIII']


,date,city,station_code,station_name,region,PM25,dl.PM25
0,2024-07-01 01:00:00,La Florida,RM/D12,La Florida (Acreditada),RM,34.0,ok
1,2024-07-01 02:00:00,La Florida,RM/D12,La Florida (Acreditada),RM,24.0,ok
2,2024-07-01 03:00:00,La Florida,RM/D12,La Florida (Acreditada),RM,25.0,ok
3,2024-07-01 04:00:00,La Florida,RM/D12,La Florida (Acreditada),RM,19.0,ok
4,2024-07-01 05:00:00,La Florida,RM/D12,La Florida (Acreditada),RM,16.0,ok


## 5. Data Capture Rate (DCR)

El **Data Capture Rate** mide la proporción de horas en que una estación reportó un dato válido respecto al total de horas esperadas:

$$\text{DCR} = \frac{\text{horas con dato válido (ok)}}{\text{horas totales esperadas}} \times 100$$

Es la métrica estándar para evaluar la disponibilidad operacional de una estación de monitoreo. La EPA (Environmental Protection Agency, EE.UU.) establece un umbral de **75%** de captura trimestral como requisito mínimo de completitud para que los datos de PM2.5 sean válidos en la evaluación de cumplimiento de los NAAQS (National Ambient Air Quality Standards).

> **Referencia:** [40 CFR Part 50, Appendix N](https://www.ecfr.gov/current/title-40/chapter-I/subchapter-C/part-50/appendix-Appendix%20N%20to%20Part%2050) — *Interpretation of the National Ambient Air Quality Standards for PM2.5*, Secciones 4.1(b) y 4.2(b).

In [12]:
def compute_dcr(df: pd.DataFrame, parameter: str) -> pd.DataFrame:
    """Calcula el DCR y el desglose de status por estación.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame retornado por get_data / get_data_async.
    parameter : str
        Nombre del parámetro (e.g. 'PM25').

    Returns
    -------
    pd.DataFrame
        Una fila por estación con columnas: station_name, region,
        total_hours, ok, empty, download_error, curated, dcr_pct.
    """
    dl_col = f"dl.{parameter}"
    if dl_col not in df.columns:
        raise ValueError(f"Column '{dl_col}' not found in DataFrame")

    group_cols = ["station_name", "region"]
    grouped = df.groupby(group_cols)[dl_col]

    # Conteo total y por status
    total = grouped.count().rename("total_hours")
    status_counts = (
        df.groupby(group_cols + [dl_col])
        .size()
        .unstack(fill_value=0)
    )

    result = status_counts.join(total).reset_index()

    # Asegurar que existan todas las columnas de status
    for status in ["ok", "empty", "download_error", "curated"]:
        if status not in result.columns:
            result[status] = 0

    result["dcr_pct"] = (result["ok"] / result["total_hours"] * 100).round(1)

    return result.sort_values("dcr_pct", ascending=False).reset_index(drop=True)

### 5a. DCR por estación — Región Metropolitana

In [13]:
dcr_all = compute_dcr(df_regions, "PM25")
dcr_rm = dcr_all[dcr_all["region"] == "RM"].copy()

dcr_rm

,station_name,region,download_error,empty,ok,total_hours,curated,dcr_pct
8,Talagante,RM,0,1,742,743,0,99.9
11,La Florida (Acreditada),RM,0,2,741,743,0,99.7
16,Parque O'Higgins,RM,0,3,740,743,0,99.6
17,Pudahuel (Acreditada),RM,0,3,740,743,0,99.6
22,Cerrillos II,RM,0,3,740,743,0,99.6
23,Quilicura,RM,0,3,740,743,0,99.6
25,Cerro Navia,RM,0,4,739,743,0,99.5
40,Puente Alto,RM,0,27,716,743,0,96.4


In [14]:
EPA_THRESHOLD = 75  # %

fig = px.bar(
    dcr_rm.sort_values("dcr_pct"),
    x="dcr_pct",
    y="station_name",
    orientation="h",
    title="DCR de PM2.5 por estación — Región Metropolitana (Julio 2024)",
    labels={"dcr_pct": "DCR (%)", "station_name": "Estación"},
    text="dcr_pct",
    color="dcr_pct",
    color_continuous_scale=["#e74c3c", "#f39c12", "#2ecc71"],
    range_color=[0, 100],
)

# Línea umbral EPA 75%
fig.add_vline(
    x=EPA_THRESHOLD, line_dash="dash", line_color="black",
    annotation_text=f"Umbral EPA ({EPA_THRESHOLD}%)",
    annotation_position="top",
)

fig.update_layout(height=400, coloraxis_showscale=False)
fig.show()

### 5b. DCR por estación — todas las regiones

In [15]:
# Ordenar por región y DCR para el gráfico
dcr_sorted = dcr_all.sort_values(["region", "dcr_pct"], ascending=[True, True])

fig = px.bar(
    dcr_sorted,
    x="dcr_pct",
    y="station_name",
    orientation="h",
    color="region",
    title="DCR de PM2.5 por estación y región (Julio 2024)",
    labels={"dcr_pct": "DCR (%)", "station_name": "Estación", "region": "Región"},
    text="dcr_pct",
    height=max(400, len(dcr_sorted) * 25),
)

fig.add_vline(
    x=EPA_THRESHOLD, line_dash="dash", line_color="black",
    annotation_text=f"Umbral EPA ({EPA_THRESHOLD}%)",
    annotation_position="top",
)

fig.update_layout(yaxis={"categoryorder": "array", "categoryarray": dcr_sorted["station_name"].tolist()})
fig.show()

### 5c. Resumen DCR por región

In [16]:
region_summary = (
    dcr_all.groupby("region")
    .agg(
        n_estaciones=("station_name", "count"),
        dcr_promedio=("dcr_pct", "mean"),
        dcr_mediana=("dcr_pct", "median"),
        dcr_min=("dcr_pct", "min"),
        dcr_max=("dcr_pct", "max"),
        estaciones_sobre_75=(
            "dcr_pct", lambda x: (x >= EPA_THRESHOLD).sum()
        ),
    )
    .round(1)
    .reset_index()
)

region_summary["pct_sobre_75"] = (
    region_summary["estaciones_sobre_75"] / region_summary["n_estaciones"] * 100
).round(1)

region_summary

,region,n_estaciones,dcr_promedio,dcr_mediana,dcr_min,dcr_max,estaciones_sobre_75,pct_sobre_75
0,II,11,61.9,96.1,0.0,99.7,7,63.6
1,RM,8,99.2,99.6,96.4,99.9,8,100.0
2,V,26,49.1,44.6,0.0,100.0,13,50.0
3,VIII,26,82.1,98.8,0.0,100.0,22,84.6


In [17]:
fig = px.box(
    dcr_all,
    x="region",
    y="dcr_pct",
    color="region",
    points="all",
    hover_data=["station_name"],
    title="Distribución del DCR de PM2.5 por región (Julio 2024)",
    labels={"dcr_pct": "DCR (%)", "region": "Región"},
)

fig.add_hline(
    y=EPA_THRESHOLD, line_dash="dash", line_color="black",
    annotation_text=f"Umbral EPA ({EPA_THRESHOLD}%)",
)

fig.update_layout(height=500, showlegend=False)
fig.show()

### 5d. Desglose de no-captura

El complemento del DCR (datos faltantes) puede deberse a distintas causas. Las columnas `dl.*` permiten desglosar:

- **empty**: la estación reportó la hora pero sin valor → posible falla del sensor
- **download_error**: la descarga HTTP falló → problema de red o servidor SINCA
- **curated**: el dato existía pero fue eliminado por inconsistencia física (e.g., PM2.5 > PM10)

In [18]:
# Calcular porcentajes por status
status_cols = ["ok", "empty", "download_error", "curated"]
dcr_pct_breakdown = dcr_all[["station_name", "region", "total_hours"] + status_cols].copy()

for col in status_cols:
    dcr_pct_breakdown[f"{col}_pct"] = (
        dcr_pct_breakdown[col] / dcr_pct_breakdown["total_hours"] * 100
    ).round(1)

# Preparar datos para stacked bar
pct_cols = [f"{c}_pct" for c in status_cols]
melted = dcr_pct_breakdown.melt(
    id_vars=["station_name", "region"],
    value_vars=pct_cols,
    var_name="status",
    value_name="pct",
)
melted["status"] = melted["status"].str.replace("_pct", "")

# Orden por DCR descendente
station_order = dcr_all.sort_values("dcr_pct", ascending=True)["station_name"].tolist()

color_map = {
    "ok": "#2ecc71",
    "empty": "#f39c12",
    "download_error": "#e74c3c",
    "curated": "#9b59b6",
}

fig = px.bar(
    melted,
    x="pct",
    y="station_name",
    color="status",
    orientation="h",
    title="Desglose de disponibilidad PM2.5 por estación (Julio 2024)",
    labels={"pct": "% de horas", "station_name": "Estación", "status": "Status"},
    color_discrete_map=color_map,
    category_orders={"status": status_cols, "station_name": station_order},
    height=max(400, len(dcr_all) * 25),
)

fig.update_layout(barmode="stack", xaxis_range=[0, 100])
fig.show()

In [19]:
# Desglose agregado por región
region_breakdown = (
    dcr_all.groupby("region")[status_cols]
    .sum()
    .reset_index()
)
region_breakdown["total"] = region_breakdown[status_cols].sum(axis=1)

for col in status_cols:
    region_breakdown[f"{col}_pct"] = (
        region_breakdown[col] / region_breakdown["total"] * 100
    ).round(1)

melted_region = region_breakdown.melt(
    id_vars=["region"],
    value_vars=pct_cols,
    var_name="status",
    value_name="pct",
)
melted_region["status"] = melted_region["status"].str.replace("_pct", "")

fig = px.bar(
    melted_region,
    x="region",
    y="pct",
    color="status",
    title="Desglose de disponibilidad PM2.5 agregado por región (Julio 2024)",
    labels={"pct": "% de horas", "region": "Región", "status": "Status"},
    color_discrete_map=color_map,
    category_orders={"status": status_cols},
    text="pct",
)

fig.update_layout(barmode="stack", yaxis_range=[0, 100], height=450)
fig.show()

### 5e. Mapa de estaciones coloreadas por DCR

In [20]:
# Unir DCR con coordenadas
dcr_geo = dcr_all.merge(
    stations[["station_name", "latitude", "longitude", "city"]],
    on="station_name",
    how="left",
)

dcr_geo["cumple_epa"] = dcr_geo["dcr_pct"].apply(
    lambda x: f"✅ ≥{EPA_THRESHOLD}%" if x >= EPA_THRESHOLD else f"❌ <{EPA_THRESHOLD}%"
)

fig = px.scatter_mapbox(
    dcr_geo,
    lat="latitude",
    lon="longitude",
    color="dcr_pct",
    size="dcr_pct",
    hover_name="station_name",
    hover_data=["region", "city", "dcr_pct", "cumple_epa"],
    color_continuous_scale=["#e74c3c", "#f39c12", "#2ecc71"],
    range_color=[0, 100],
    title="DCR de PM2.5 por estación (Julio 2024)",
    zoom=4,
    height=600,
    size_max=15,
)

fig.update_layout(mapbox_style="open-street-map")
fig.show()

## 6. Datos climáticos (DMC)

La librería también permite descargar datos meteorológicos de la DMC. Veamos un ejemplo con temperatura y humedad.

In [21]:
ccd = ChileClimateData()
climate_stations = ccd.stations_table

print(f"Estaciones DMC: {len(climate_stations)}")
print(f"Parámetros disponibles: {ccd.available_parameters}")
climate_stations[["Nombre", "Código Nacional", "Region"]].head(10)

Estaciones DMC: 47
Parámetros disponibles: ['Temperatura', 'PuntoRocio', 'Humedad', 'Viento', 'PresionQFE', 'PresionQFF']


,Nombre,Código Nacional,Region
0,Chacalluta Arica Ap.,180005,XV
1,Diego Aracena Iquique Ap.,200006,I
2,El Loa Calama Ad.,220002,II
3,Cerro Moreno Antofagasta Ap.,230001,II
4,Mataveri Isla de Pascua Ap.,270001,V
5,Desierto de Atacama Caldera Ad.,270008,III
6,La Florida La Serena Ad.,290004,VI
7,Viña del Mar Ad. (Torquemada),320041,V
8,Los Libertadores,320051,V
9,Rodelillo Ad.,330007,V


In [23]:
# Quinta Normal, Santiago — estación meteorológica de referencia
df_climate = ccd.get_data(
    stations="330020",
    parameters=["Temperatura", "Humedad"],
    start=datetime(2024, 7, 1, 0, 0, 0),
    end=datetime(2024, 7, 31, 23, 0, 0),
)

print(f"Shape: {df_climate.shape}")
print(f"Columnas: {df_climate.columns.tolist()}")
df_climate.head()

Shape: (744, 9)
Columnas: ['date', 'Ts', 'dl.Ts', 'HR', 'dl.HR', 'Nombre', 'CodigoNacional', 'Latitud', 'Longitud']


,date,Ts,dl.Ts,HR,dl.HR,Nombre,CodigoNacional,Latitud,Longitud
0,2024-07-01 00:00:00,8.1,ok,86.0,ok,Quinta Normal Santiago,330020,-33.445,-70.67778
1,2024-07-01 01:00:00,7.1,ok,94.8,ok,Quinta Normal Santiago,330020,-33.445,-70.67778
2,2024-07-01 02:00:00,6.7,ok,96.9,ok,Quinta Normal Santiago,330020,-33.445,-70.67778
3,2024-07-01 03:00:00,6.2,ok,97.0,ok,Quinta Normal Santiago,330020,-33.445,-70.67778
4,2024-07-01 04:00:00,5.3,ok,98.8,ok,Quinta Normal Santiago,330020,-33.445,-70.67778


In [24]:
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    subplot_titles=("Temperatura (°C)", "Humedad Relativa (%)"),
    vertical_spacing=0.08,
)

if "Ts" in df_climate.columns:
    fig.add_trace(
        go.Scatter(x=df_climate["date"], y=df_climate["Ts"],
                   mode="lines", name="Temperatura",
                   line=dict(color="#e74c3c", width=1)),
        row=1, col=1,
    )

if "HR" in df_climate.columns:
    fig.add_trace(
        go.Scatter(x=df_climate["date"], y=df_climate["HR"],
                   mode="lines", name="Humedad",
                   line=dict(color="#3498db", width=1)),
        row=2, col=1,
    )

fig.update_layout(
    title="Quinta Normal, Santiago — Julio 2024",
    height=500,
)
fig.show()

## 7. Conclusión

En este notebook cubrimos:

- **Exploración** de las estaciones SINCA/DMC disponibles
- **Descarga sincrónica** de datos para una estación individual
- **Descarga asincrónica** masiva por región
- **Data Capture Rate (DCR)** como métrica de disponibilidad, con desglose por estación y región
- **Desglose de no-captura** (`empty`, `download_error`, `curated`) para diagnosticar causas de datos faltantes
- **Datos climáticos** del DMC con temperatura y humedad